TOPICS
- satellite tracker
- github activity
  - follow activity on a few interesting repo's 
- reddit activity
- 

# StarLink Tracker with Delta Lake

My house lat/long:
37.668729, -8.374958

APIkey= B7FTK6-VRVYCB-U8NYQM-59J8

## Questions to ask
0. Can I predict when my Starlink internet will be fastest?

1. Which Starlink satellites cross overhead in a day?
- get list of passing satellites (full sky, 90*) every minute
- check for duplicates
- add non-duplicates to Delta tables

In [1]:
from deltalake import DeltaTable, write_deltalake
import pandas as pd
import requests

## Get Raw Data from REST API

In [2]:
api_url = "https://api.n2yo.com/rest/v1/satellite/above/37.668729/-8.374958/125/90/52&apiKey=B7FTK6-VRVYCB-U8NYQM-59J8"
response = requests.get(api_url)

In [3]:
json = response.json()
raw_data = json.get('above')

In [4]:
df = pd.DataFrame.from_dict(raw_data)

In [5]:
df.head()

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt
0,44967,STARLINK-1089,2020-001BF,2020-01-07,18.5800,-13.0478,547.1382
1,45186,STARLINK-1224,2020-012J,2020-02-17,47.2555,-24.4661,551.5441
2,45212,STARLINK-1231,2020-012AL,2020-02-17,52.7237,2.2168,552.5686
3,45214,STARLINK-1233,2020-012AN,2020-02-17,51.1724,-13.5519,364.9616
4,45235,STARLINK-1246,2020-012BK,2020-02-17,53.0640,5.6968,552.7401


In [6]:
len(df)

305

In [7]:
# write to Delta Lake
delta_path= "/Users/rpelgrim/Desktop/delta-data/starlink-overhead-test"
write_deltalake(delta_path, df)

In [8]:
df = DeltaTable(delta_path).to_pandas()
df = df.drop(columns=["satalt"])

In [9]:
write_deltalake(
    delta_path, 
    df, 
    mode="overwrite",
    schema_mode="overwrite",
)

In [11]:
df = DeltaTable(delta_path).to_pandas()
df

,satid,satname,intDesignator,launchDate,satlat,satlng
0,44967,STARLINK-1089,2020-001BF,2020-01-07,18.5800,-13.0478
1,45186,STARLINK-1224,2020-012J,2020-02-17,47.2555,-24.4661
2,45212,STARLINK-1231,2020-012AL,2020-02-17,52.7237,2.2168
3,45214,STARLINK-1233,2020-012AN,2020-02-17,51.1724,-13.5519
4,45235,STARLINK-1246,2020-012BK,2020-02-17,53.0640,5.6968
...,...,...,...,...,...,...
300,59537,STARLINK-31737,2024-074D,2024-04-18,41.8628,-8.1295
301,59551,STARLINK-31699,2024-074T,2024-04-18,38.2637,-23.7096
302,59553,STARLINK-31723,2024-074V,2024-04-18,43.1585,8.5207
303,59571,STARLINK-31606,2024-076H,2024-04-23,27.0602,-20.8112


## Read Data for Analysis

In [8]:
# get existing data on disk
df = DeltaTable(delta_path).to_pandas()
satids = df.satid.to_list()
len(satids)

301

In [9]:
# import time
# time.sleep(60)

# fetch new data from REST
response = requests.get(api_url)
json = response.json()
raw_data = json.get('above')
df_new = pd.DataFrame.from_dict(raw_data)

In [10]:
len(df_new)

299

In [11]:
satids_new = df_new.satid.to_list()

In [12]:
# get new satids
s = set(satids)
diff = [x for x in satids_new if x not in s]

In [13]:
len(diff)

14

In [14]:
df_new[df_new['satid'].isin(diff)]

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt
35,47362,STARLINK-2076,2021-005P,2021-01-20,29.0760,-31.7270,548.1891
40,47550,STARLINK-1909,2021-009C,2021-02-04,53.1207,-30.8427,552.8330
42,47554,STARLINK-1953,2021-009G,2021-02-04,45.2185,-37.3988,550.9189
81,51151,STARLINK-3361,2022-005AZ,2022-01-19,44.7879,-36.6645,543.6648
115,52483,STARLINK-3875,2022-049AJ,2022-05-06,36.9045,-34.8567,541.8553
123,53008,STARLINK-4275,2022-076Y,2022-07-07,16.6489,-10.9570,539.5514
156,54203,STARLINK-5247,2022-141AY,2022-10-28,53.3567,-32.2677,545.4623
187,56296,STARLINK-30089,2023-056L,2023-04-19,23.9819,-28.0293,559.2520
189,56317,STARLINK-6038,2023-058A,2023-04-27,24.8464,13.1971,565.4865
196,56451,STARLINK-5985,2023-064D,2023-05-10,58.4047,-20.5563,583.0110


In [21]:
df_new_noduplicates = df_new[df_new['satid'].isin(diff)]

In [22]:
df_new_noduplicates.head()

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt
35,47362,STARLINK-2076,2021-005P,2021-01-20,29.0760,-31.7270,548.1891
40,47550,STARLINK-1909,2021-009C,2021-02-04,53.1207,-30.8427,552.8330
42,47554,STARLINK-1953,2021-009G,2021-02-04,45.2185,-37.3988,550.9189
81,51151,STARLINK-3361,2022-005AZ,2022-01-19,44.7879,-36.6645,543.6648
115,52483,STARLINK-3875,2022-049AJ,2022-05-06,36.9045,-34.8567,541.8553


In [23]:
len(df_new_noduplicates)

14

In [24]:
df_new_noduplicates.reset_index(drop=True, inplace=True)

In [25]:
df_new_noduplicates.head()

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt
0,47362,STARLINK-2076,2021-005P,2021-01-20,29.0760,-31.7270,548.1891
1,47550,STARLINK-1909,2021-009C,2021-02-04,53.1207,-30.8427,552.8330
2,47554,STARLINK-1953,2021-009G,2021-02-04,45.2185,-37.3988,550.9189
3,51151,STARLINK-3361,2022-005AZ,2022-01-19,44.7879,-36.6645,543.6648
4,52483,STARLINK-3875,2022-049AJ,2022-05-06,36.9045,-34.8567,541.8553


In [26]:
write_deltalake(
    delta_path, 
    df_new_noduplicates, 
    mode="append",
#    schema_mode="merge",
#    engine="rust",
)

In [21]:
df = DeltaTable(delta_path).to_pandas()
df

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt,__index_level_0__
0,46077,STARLINK-1559,2020-055BC,2020-08-07,17.5327,-4.7633,547.2574,12.0
1,46169,STARLINK-1628,2020-057BE,2020-08-18,47.9304,-34.0059,551.5634,17.0
2,46770,STARLINK-1932,2020-074AH,2020-10-24,51.5902,-28.4968,552.5571,27.0
3,47354,STARLINK-2046,2021-005F,2021-01-20,47.4596,-33.3001,551.5808,33.0
4,47373,STARLINK-2092,2021-005AA,2021-01-20,27.0990,-30.9297,547.9703,37.0
...,...,...,...,...,...,...,...,...
327,59545,STARLINK-31582,2024-074M,2024-04-18,42.6522,8.1876,447.9637,NaN
328,59555,STARLINK-31730,2024-074X,2024-04-18,39.9037,-7.9093,447.3430,NaN
329,59568,STARLINK-31471,2024-076E,2024-04-23,36.7758,8.9535,446.7919,NaN
330,59582,STARLINK-31701,2024-076U,2024-04-23,30.9003,-4.2081,445.1565,NaN


- OK, this works. Not ideal but OK
- now i need to drop that extra index col
- doesn't seem straightforward in delta-rs

see [delta blog](https://delta.io/blog/2022-08-29-delta-lake-drop-column) for Spark

In [ ]:
# OK it looks like 15-minutes is too long, they're all new
# is that possible?

In [ ]:
# I actually don't need satlng satlat and satalt

## Script

In [1]:
from deltalake import DeltaTable, write_deltalake
import pandas as pd
import requests
import time

api_url = "https://api.n2yo.com/rest/v1/satellite/above/37.668729/-8.374958/125/90/52&apiKey=B7FTK6-VRVYCB-U8NYQM-59J8"
response = requests.get(api_url)
json = response.json()
raw_data = json.get('above')
df = pd.DataFrame.from_dict(raw_data)

# write to Delta Lake
delta_path= "/Users/rpelgrim/Desktop/delta-data/starlink-overhead"
write_deltalake(delta_path, df)
print("First write complete")

df = DeltaTable(delta_path).to_pandas()
satids = df.satid.to_list()
print(f'Delta Lake now has {len(satids)} sat_ids')

for i in range(4):
    time.sleep(60)
    
    # fetch new data from REST
    print('Getting new data')
    response = requests.get(api_url)
    json = response.json()
    raw_data = json.get('above')
    df_new = pd.DataFrame.from_dict(raw_data)
    satids_new = df_new.satid.to_list()
    
    # get new satids
    s = set(satids)
    diff = [x for x in satids_new if x not in s]
    
    print(f'There are {len(diff)} new sat_ids')
    
    df_new_noduplicates = df_new[df_new['satid'].isin(diff)]
    df_new_noduplicates.reset_index(drop=True, inplace=True)
    
    # append new satellites to Delta Lake
    write_deltalake(
        delta_path, 
        df_new_noduplicates, 
        mode="append",
    )
    print(f"Write #{i+2} completed.")

First write complete
Delta Lake now has 296 sat_ids
Getting new data
There are 24 new sat_ids
Write #2 completed.
Getting new data
There are 55 new sat_ids
Write #3 completed.
Getting new data
There are 85 new sat_ids
Write #4 completed.
Getting new data
There are 119 new sat_ids
Write #5 completed.


In [4]:
# verify unique ids
df = DeltaTable(delta_path).to_pandas()
satids = df.satid.to_list()
len(satids)

NameError: name 'delta_path' is not defined

In [3]:
df

,satid,satname,intDesignator,launchDate,satlat,satlng,satalt
0,44969,STARLINK-1092,2020-001BH,2020-01-07,35.2449,-24.8275,549.1750
1,45184,STARLINK-1205,2020-012G,2020-02-17,50.3410,-28.4810,551.9786
2,45185,STARLINK-1216,2020-012H,2020-02-17,46.0188,-22.1236,551.2690
3,45230,STARLINK-1227,2020-012BE,2020-02-17,52.7856,-32.9360,552.6389
4,45554,STARLINK-1325,2020-025Z,2020-04-22,22.8969,-14.5216,547.4809
...,...,...,...,...,...,...,...
574,59554,STARLINK-31724,2024-074W,2024-04-18,32.1192,-32.4849,445.6289
575,59566,STARLINK-31332,2024-076C,2024-04-23,33.9451,-1.5583,446.2017
576,59574,STARLINK-31751,2024-076L,2024-04-23,24.3154,-18.5873,443.5400
577,59581,STARLINK-31690,2024-076T,2024-04-23,39.0738,12.5112,447.1609


In [6]:
len(df.satid.unique())

416

In [5]:
# this is not quite working yet
# some duplicates